# 01 - Análisis y ETL con Pandas

Este notebook realiza la exploración, limpieza y el proceso ETL del dataset de videojuegos usando Pandas y carga el resultado a una base de datos SQLite (`warehouse/warehouse_pandas.db`).

## 1. Configuración Inicial

Se importan las librerías necesarias para el análisis: Pandas para manipulación de datos, SQLAlchemy para conexión a bases de datos, NumPy para operaciones numéricas, y otras utilidades. Se verifica que se carguen correctamente mostrando sus versiones.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import sqlalchemy

# -------------------------------
# Mensajes de información
# -------------------------------

print("Librerías cargadas correctamente:")
print(" - os:", os)
print(" - re (expresiones regulares):", re)
print(" - numpy:", np.__version__)
print(" - pandas:", pd.__version__)
print(" - sqlalchemy:", sqlalchemy.__version__)
print("\nPreparado para cargar y procesar el dataset de videojuegos.\n")

## 2. Carga del Dataset

Se carga el dataset de videojuegos desde el archivo CSV ubicado en `data/videogames.csv` usando `pd.read_csv()`. Se muestra información básica como dimensiones del dataset, tipos de datos y las primeras filas para confirmar la carga correcta.

In [ ]:
# Se carga el CSV desde la carpeta /data
raw_df = pd.read_csv('../data/videogames.csv')

# Mensaje informativo y revisión inicial
print("Dataset cargado correctamente desde '../data/videogames.csv'")
print(f"Dimensiones del dataset: {raw_df.shape[0]} filas x {raw_df.shape[1]} columnas")
print("Primeras 5 filas del dataset:")
print(raw_df.head())

## 3. Exploración Inicial del Dataset

Se realiza una exploración inicial del dataset usando `info()` y `describe()`. Se muestran estadísticas descriptivas, tipos de datos, valores únicos y presencia de nulos para entender la estructura y calidad de los datos antes de proceder con la limpieza.

In [ ]:
# Inspección inicial de los datos
print("===== Información general del dataset =====")
raw_df.info() # Muestra tipos de datos, nulos y número de filas/columnas

print("\n===== Estadísticas descriptivas =====")
print(raw_df.describe(include="all").T) # Muestra estadísticas descriptivas de todas las columnas (numéricas y categóricas)

## 4. Limpieza de Datos

Se eliminan filas duplicadas con `drop_duplicates()` y se tratan valores faltantes: nulos en columnas numéricas (como `copies_sold_millions` y `revenue_millions_usd`) se imputan con la media después de convertir formatos especiales (ej. "1.5M" a números), y en categóricas se rellenan con "Unknown".

In [ ]:
print("===== 1️ Eliminación de duplicados =====")
clean_df = raw_df.drop_duplicates().copy()
print(f"Filas después de eliminar duplicados: {clean_df.shape[0]}")

# -------------------------------
# Identificación de tipos de columnas
# -------------------------------
numeric_cols = clean_df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = clean_df.select_dtypes(include=["object"]).columns.tolist()

# --------------------------------------------------
# Limpieza de columnas numéricas problemáticas
# --------------------------------------------------
def parse_numeric(val):
    if pd.isna(val):
        return np.nan
    if isinstance(val, str):
        val = val.strip().replace(",", "").replace("$", "").upper()
        if val in ["UNKNOWN", "?"]:
            return np.nan
        if val.endswith("M"):
            try: return float(val[:-1]) * 1e6
            except: return np.nan
        if val.endswith("B"):
            try: return float(val[:-1]) * 1e9
            except: return np.nan
    try:
        return float(val)
    except:
        return np.nan

print("\n===== 2 Limpieza de columnas de ventas/ingresos =====")
for col in ["copies_sold_millions", "revenue_millions_usd"]:
    if col in clean_df.columns:
        # Aplicamos la función de limpieza
        clean_df[col] = clean_df[col].apply(parse_numeric)
        # Convertimos a tipo float explícitamente
        clean_df[col] = pd.to_numeric(clean_df[col], errors='coerce')
        # Rellenamos los NaN con la media
        clean_df[col] = clean_df[col].fillna(clean_df[col].mean())
        print(f"Columna '{col}' limpiada. Valores nulos restantes: {clean_df[col].isna().sum()}")

# --------------------------------------------------
# Imputación de columnas categóricas
# --------------------------------------------------
cat_cols = clean_df.select_dtypes(include=["object"]).columns.tolist()
print("\n===== 3 Imputación de columnas categóricas =====")
for col in cat_cols:
    clean_df[col] = clean_df[col].fillna("Unknown")
    print(f"Columna '{col}' rellenada con 'Unknown'. Valores nulos restantes: {clean_df[col].isna().sum()}")

# Resumen final
print("\n===== Resumen final de limpieza =====")
print(clean_df.isna().sum())
print(f"Dimensiones del dataset limpio: {clean_df.shape[0]} filas x {clean_df.shape[1]} columnas")

## 5. Normalización y Transformaciones

Se normalizan los nombres de las columnas a formato snake_case (minúsculas, guiones bajos) para consistencia usando operaciones de string de Pandas. Se verifican las columnas requeridas para el modelo dimensional antes de construir las tablas.

In [ ]:
print("===== Normalización de nombres de columnas =====")
print("Columnas antes de normalizar:")
print(list(raw_df.columns))

# Convertimos nombres a snake_case para facilidad de uso
clean_df.columns = (
    clean_df.columns
    .str.strip()             # Elimina espacios al inicio y final
    .str.lower()             # Convierte a minúsculas
    .str.replace(" ", "_", regex=False)  # Reemplaza espacios por '_'
    .str.replace("-", "_", regex=False)  # Reemplaza '-' por '_'
)

print("\nColumnas después de normalizar a snake_case:")
print(list(clean_df.columns))

## 6. Creación de Dimensiones

Se crean las tablas de dimensiones del modelo en estrella: `dim_game`, `dim_platform`, `dim_developer`, `dim_publisher` y `dim_year`. Se obtienen valores únicos con `drop_duplicates()`, se ordenan y se asignan IDs autoincrementales usando NumPy.

In [ ]:
print("===== Validación de columnas requeridas =====")
required_cols = ["name", "genre", "platform", "developer", "publisher", "year"]
for c in required_cols:
    if c not in clean_df.columns:
        raise ValueError(f"Columna requerida no encontrada en el CSV: {c}")
print("Todas las columnas requeridas están presentes.\n")

# -------------------------------
# Dimensión: Videojuegos
# -------------------------------
dim_game = clean_df[["name", "genre"]].drop_duplicates().reset_index(drop=True)
dim_game["id_game"] = np.arange(1, len(dim_game) + 1)
print("Dimensión 'dim_game' creada:")
print(f"Filas: {dim_game.shape[0]}")
print(dim_game.head())

# -------------------------------
# Dimensión: Plataformas
# -------------------------------
dim_platform = clean_df[["platform"]].drop_duplicates().reset_index(drop=True)
dim_platform["id_platform"] = np.arange(1, len(dim_platform) + 1)
print("\nDimensión 'dim_platform' creada:")
print(f"Filas: {dim_platform.shape[0]}")
print(dim_platform.head())

# -------------------------------
# Dimensión: Desarrolladores
# -------------------------------
dim_developer = clean_df[["developer"]].drop_duplicates().reset_index(drop=True)
dim_developer["id_developer"] = np.arange(1, len(dim_developer) + 1)
print("\nDimensión 'dim_developer' creada:")
print(f"Filas: {dim_developer.shape[0]}")
print(dim_developer.head())

# -------------------------------
# Dimensión: Publisher
# -------------------------------
dim_publisher = clean_df[["publisher"]].drop_duplicates().reset_index(drop=True)
dim_publisher["id_publisher"] = np.arange(1, len(dim_publisher) + 1)
print("\nDimensión 'dim_publisher' creada:")
print(f"Filas: {dim_publisher.shape[0]}")
print(dim_publisher.head())

# -------------------------------
# Dimensión: Año
# -------------------------------
dim_year = clean_df[["year"]].drop_duplicates().reset_index(drop=True)
dim_year["id_year"] = np.arange(1, len(dim_year) + 1)
print("\nDimensión 'dim_year' creada:")
print(f"Filas: {dim_year.shape[0]}")
print(dim_year.head())

## 7. Construcción de la Tabla de Hechos

Se construye la tabla de hechos `fact_sales` mediante merges (joins) entre el dataset limpio y las dimensiones usando `merge()` con `how='left'`. Se incluyen las métricas `copies_sold_millions` y `revenue_millions_usd` junto con las claves foráneas de las dimensiones.

In [ ]:
print("===== Unión de dimensiones para crear la tabla de hechos =====")
# Se unen todas las dimensiones al dataset original para generar la tabla de hechos
fact = clean_df.merge(dim_game, on=["name", "genre"], how="left")
fact = fact.merge(dim_platform, on=["platform"], how="left")
fact = fact.merge(dim_developer, on=["developer"], how="left")
fact = fact.merge(dim_publisher, on=["publisher"], how="left")
fact = fact.merge(dim_year, on=["year"], how="left")
print("Unión completada. Dimensiones del dataset de hechos:", fact.shape)

# -------------------------------
# Selección de columnas métricas
# -------------------------------
# Verificamos qué columnas de ventas/ingresos existen en el dataset
value_cols = [c for c in ["copies_sold_millions", "revenue_millions_usd"] if c in fact.columns]
if not value_cols:
    raise ValueError("No se encontraron columnas de ventas/ingresos en el CSV.")
print("Columnas métricas seleccionadas para la tabla de hechos:", value_cols)

# -------------------------------
# Tabla de hechos final
# -------------------------------
fact_sales = fact[["id_game", "id_platform", "id_developer", "id_publisher", "id_year"] + value_cols].copy()
print("\nTabla de hechos 'fact_sales' creada:")
print(f"Dimensiones: {fact_sales.shape[0]} filas x {fact_sales.shape[1]} columnas")
print("Primeros registros de la tabla de hechos:")
print(fact_sales.head())


## 8. Carga en SQLite

Se carga el Data Warehouse en SQLite: se crea la conexión con SQLAlchemy usando `create_engine()`, se crea la carpeta `warehouse` si no existe, y se insertan las tablas de dimensiones y hechos usando `to_sql()` con `if_exists='replace'` para sobrescribir si existen.

In [ ]:
print("===== 1 Creación de carpeta y configuración de la base de datos =====")
# Creamos la carpeta warehouse si no existe
os.makedirs("../warehouse", exist_ok=True)
DB_PATH = "../warehouse/warehouse_pandas.db"
DB_URL = f"sqlite:///{DB_PATH}"
print(f"Base de datos SQLite se guardará en: {DB_PATH}")

# Creamos el engine para SQLite
engine = sqlalchemy.create_engine(DB_URL)

# -------------------------------
# Guardado de tablas dimensionales y tabla de hechos
# -------------------------------
print("\n===== 2️ Guardado de tablas en SQLite =====")
with engine.begin() as conn:
    dim_game.to_sql("dim_game", conn, if_exists="replace", index=False)
    print(" - dim_game cargada")
    dim_platform.to_sql("dim_platform", conn, if_exists="replace", index=False)
    print(" - dim_platform cargada")
    dim_developer.to_sql("dim_developer", conn, if_exists="replace", index=False)
    print(" - dim_developer cargada")
    dim_publisher.to_sql("dim_publisher", conn, if_exists="replace", index=False)
    print(" - dim_publisher cargada")
    dim_year.to_sql("dim_year", conn, if_exists="replace", index=False)
    print(" - dim_year cargada")
    fact_sales.to_sql("fact_sales", conn, if_exists="replace", index=False)
    print(" - fact_sales cargada")

print("\n Todas las tablas fueron cargadas correctamente en SQLite.")

## 9. Consultas y Validación

Se ejecutan consultas SQL de ejemplo sobre el Data Warehouse para validar los datos: se calcula el top 10 de géneros por ventas usando `pd.read_sql()` con joins entre `fact_sales` y `dim_game`, y se muestra el resultado para confirmar que el DW funciona correctamente.

In [ ]:
print("===== Consulta: Top 10 géneros por ventas =====")

# Consulta SQL para obtener los géneros con mayores ventas en millones de copias
query = """
SELECT g.genre, SUM(f.copies_sold_millions) AS total_sales
FROM fact_sales f
JOIN dim_game g ON f.id_game = g.id_game
GROUP BY g.genre
ORDER BY total_sales DESC
LIMIT 10;
"""
print("Consulta SQL ejecutada:\n", query)

# Ejecutamos la consulta y la cargamos en un DataFrame de Pandas
with engine.connect() as conn:
    top_genres = pd.read_sql(query, conn)

# Mostramos resultados
print("\n===== Resultado: Top 10 géneros por ventas =====")
print(top_genres)
